In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 800 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
%time hist1 = train_model(env, model, total_episodes=7000)

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -21.000000. running mean: -21.000000
episode 2.000000, reward total was -21.000000. running mean: -21.000000
episode 3.000000, reward total was -21.000000. running mean: -21.000000
episode 4.000000, reward total was -21.000000. running mean: -21.000000
episode 5.000000, reward total was -21.000000. running mean: -21.000000
episode 6.000000, reward total was -21.000000. running mean: -21.000000
episode 7.000000, reward total was -21.000000. running mean: -21.000000
episode 8.000000, reward total was -20.000000. running mean: -20.990000
episode 9.000000, reward total was -21.000000. running mean: -20.990100
episode 10.000000, reward total was -21.000000. running mean: -20.990199
episode 11.000000, reward total was -21.000000. running mean: -20.990297
episode 12.000000, reward total was -21.000000. running mean: -20.990394
episode 13.000000, reward total was -21.000000. running mean: -20.990490
episode 14.000000, reward total was -21.000000. running mean

episode 114.000000, reward total was -21.000000. running mean: -20.621898
episode 115.000000, reward total was -20.000000. running mean: -20.615679
episode 116.000000, reward total was -21.000000. running mean: -20.619522
episode 117.000000, reward total was -20.000000. running mean: -20.613327
episode 118.000000, reward total was -21.000000. running mean: -20.617194
episode 119.000000, reward total was -21.000000. running mean: -20.621022
episode 120.000000, reward total was -20.000000. running mean: -20.614811
episode 121.000000, reward total was -18.000000. running mean: -20.588663
episode 122.000000, reward total was -20.000000. running mean: -20.582777
episode 123.000000, reward total was -21.000000. running mean: -20.586949
episode 124.000000, reward total was -21.000000. running mean: -20.591079
episode 125.000000, reward total was -21.000000. running mean: -20.595169
episode 126.000000, reward total was -21.000000. running mean: -20.599217
episode 127.000000, reward total was -

episode 225.000000, reward total was -21.000000. running mean: -20.497253
episode 226.000000, reward total was -21.000000. running mean: -20.502280
episode 227.000000, reward total was -20.000000. running mean: -20.497257
episode 228.000000, reward total was -21.000000. running mean: -20.502285
episode 229.000000, reward total was -19.000000. running mean: -20.487262
episode 230.000000, reward total was -20.000000. running mean: -20.482389
episode 231.000000, reward total was -20.000000. running mean: -20.477565
episode 232.000000, reward total was -20.000000. running mean: -20.472790
episode 233.000000, reward total was -21.000000. running mean: -20.478062
episode 234.000000, reward total was -20.000000. running mean: -20.473281
episode 235.000000, reward total was -21.000000. running mean: -20.478548
episode 236.000000, reward total was -21.000000. running mean: -20.483763
episode 237.000000, reward total was -21.000000. running mean: -20.488925
episode 238.000000, reward total was -

episode 336.000000, reward total was -20.000000. running mean: -20.362765
episode 337.000000, reward total was -19.000000. running mean: -20.349138
episode 338.000000, reward total was -21.000000. running mean: -20.355646
episode 339.000000, reward total was -21.000000. running mean: -20.362090
episode 340.000000, reward total was -21.000000. running mean: -20.368469
episode 341.000000, reward total was -21.000000. running mean: -20.374784
episode 342.000000, reward total was -21.000000. running mean: -20.381036
episode 343.000000, reward total was -20.000000. running mean: -20.377226
episode 344.000000, reward total was -21.000000. running mean: -20.383454
episode 345.000000, reward total was -21.000000. running mean: -20.389619
episode 346.000000, reward total was -20.000000. running mean: -20.385723
episode 347.000000, reward total was -20.000000. running mean: -20.381866
episode 348.000000, reward total was -21.000000. running mean: -20.388047
episode 349.000000, reward total was -

...

episode 6502.000000, reward total was -14.000000. running mean: -16.210531
episode 6503.000000, reward total was -20.000000. running mean: -16.248426
episode 6504.000000, reward total was -11.000000. running mean: -16.195941
episode 6505.000000, reward total was -15.000000. running mean: -16.183982
episode 6506.000000, reward total was -18.000000. running mean: -16.202142
episode 6507.000000, reward total was -16.000000. running mean: -16.200121
episode 6508.000000, reward total was -15.000000. running mean: -16.188119
episode 6509.000000, reward total was -15.000000. running mean: -16.176238
episode 6510.000000, reward total was -17.000000. running mean: -16.184476
episode 6511.000000, reward total was -19.000000. running mean: -16.212631
episode 6512.000000, reward total was -15.000000. running mean: -16.200505
episode 6513.000000, reward total was -17.000000. running mean: -16.208500
episode 6514.000000, reward total was -16.000000. running mean: -16.206415
episode 6515.000000, rewa

episode 6612.000000, reward total was -20.000000. running mean: -16.414169
episode 6613.000000, reward total was -17.000000. running mean: -16.420027
episode 6614.000000, reward total was -15.000000. running mean: -16.405827
episode 6615.000000, reward total was -19.000000. running mean: -16.431769
episode 6616.000000, reward total was -18.000000. running mean: -16.447451
episode 6617.000000, reward total was -19.000000. running mean: -16.472976
episode 6618.000000, reward total was -19.000000. running mean: -16.498247
episode 6619.000000, reward total was -18.000000. running mean: -16.513264
episode 6620.000000, reward total was -13.000000. running mean: -16.478132
episode 6621.000000, reward total was -14.000000. running mean: -16.453350
episode 6622.000000, reward total was -15.000000. running mean: -16.438817
episode 6623.000000, reward total was -18.000000. running mean: -16.454429
episode 6624.000000, reward total was -17.000000. running mean: -16.459884
episode 6625.000000, rewa

episode 6722.000000, reward total was -17.000000. running mean: -15.910050
episode 6723.000000, reward total was -18.000000. running mean: -15.930950
episode 6724.000000, reward total was -17.000000. running mean: -15.941640
episode 6725.000000, reward total was -12.000000. running mean: -15.902224
episode 6726.000000, reward total was -19.000000. running mean: -15.933202
episode 6727.000000, reward total was -13.000000. running mean: -15.903870
episode 6728.000000, reward total was -17.000000. running mean: -15.914831
episode 6729.000000, reward total was -16.000000. running mean: -15.915683
episode 6730.000000, reward total was -17.000000. running mean: -15.926526
episode 6731.000000, reward total was -7.000000. running mean: -15.837260
episode 6732.000000, reward total was -16.000000. running mean: -15.838888
episode 6733.000000, reward total was -18.000000. running mean: -15.860499
episode 6734.000000, reward total was -15.000000. running mean: -15.851894
episode 6735.000000, rewar

episode 6832.000000, reward total was -17.000000. running mean: -15.935975
episode 6833.000000, reward total was -15.000000. running mean: -15.926615
episode 6834.000000, reward total was -17.000000. running mean: -15.937349
episode 6835.000000, reward total was -17.000000. running mean: -15.947976
episode 6836.000000, reward total was -18.000000. running mean: -15.968496
episode 6837.000000, reward total was -16.000000. running mean: -15.968811
episode 6838.000000, reward total was -17.000000. running mean: -15.979123
episode 6839.000000, reward total was -14.000000. running mean: -15.959332
episode 6840.000000, reward total was -15.000000. running mean: -15.949738
episode 6841.000000, reward total was -16.000000. running mean: -15.950241
episode 6842.000000, reward total was -15.000000. running mean: -15.940739
episode 6843.000000, reward total was -15.000000. running mean: -15.931331
episode 6844.000000, reward total was -19.000000. running mean: -15.962018
episode 6845.000000, rewa

episode 6942.000000, reward total was -11.000000. running mean: -15.833992
episode 6943.000000, reward total was -17.000000. running mean: -15.845652
episode 6944.000000, reward total was -17.000000. running mean: -15.857195
episode 6945.000000, reward total was -19.000000. running mean: -15.888623
episode 6946.000000, reward total was -15.000000. running mean: -15.879737
episode 6947.000000, reward total was -14.000000. running mean: -15.860940
episode 6948.000000, reward total was -16.000000. running mean: -15.862330
episode 6949.000000, reward total was -15.000000. running mean: -15.853707
episode 6950.000000, reward total was -13.000000. running mean: -15.825170
episode 6951.000000, reward total was -19.000000. running mean: -15.856918
episode 6952.000000, reward total was -17.000000. running mean: -15.868349
episode 6953.000000, reward total was -15.000000. running mean: -15.859665
episode 6954.000000, reward total was -18.000000. running mean: -15.881069
episode 6955.000000, rewa